In [1]:
import sys
sys.path.append('E:\weather\Lib\site-packages')
# 打印环境变量
# print(sys.path)

In [55]:
import torch
# import torch.nn as nn
# import torch.nn.functional as F
#
# model = nn.Conv2d(1,1,5,2,2)
# x = torch.randn(1, 1, 160, 160)
# print(model(x).shape)
# # print(F.interpolate(x,[81,81], mode='bicubic').shape)

# x = torch.randn((2,1,10,10)).unsqueeze(0).repeat(20, 1, 1, 1, 1)
# # x = x.repeat(20,1,1,1)
# print(x.shape)
# print(x[1,:,0,0,:])
# print(x[8,:,0,0,:])
times = torch.arange(0, 20, dtype=torch.float32).unsqueeze(-1)
print(times)

tensor([[ 0.],
        [ 1.],
        [ 2.],
        [ 3.],
        [ 4.],
        [ 5.],
        [ 6.],
        [ 7.],
        [ 8.],
        [ 9.],
        [10.],
        [11.],
        [12.],
        [13.],
        [14.],
        [15.],
        [16.],
        [17.],
        [18.],
        [19.]])


In [41]:
import torch
import torch.nn as nn

# nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, padding_mode='replicate' dilation=1, groups=1, bias=True)
# model = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
# model = nn.Conv2d(1, 1, 3, stride=2, padding=1)
model = nn.Embedding(20,161*161)

for name, param in model.named_parameters():
    print(name, param.shape)

x = torch.tensor([1])
print(model(x).shape)

# criterion = nn.MSELoss()
# optimizer = torch.optim.AdamW(model.parameters(), lr=0.0005, weight_decay=0.05, betas=(0.9, 0.95))
#
# x = torch.randn(1,1,80,80)
# y = torch.randn(1,1,160,160)
# preds = model(x)
#
# loss = criterion(preds, y)
# loss.backward()
# for name, param in model.named_parameters():
#     print(name, param.grad)


weight torch.Size([20, 25921])
torch.Size([1, 25921])


In [70]:
# import os
import time
from tqdm import tqdm
from pathlib import Path

# import xarray as xr
# import numpy as np
# import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from timm.scheduler import create_scheduler
from utils.tools import getModelSize, load_model, save_model
from utils.eval import climax_pretrain_evaluate


SAVE_PATH = Path('./checkpoint/')
SAVE_PATH.mkdir(parents=True, exist_ok=True)

from climax.regional_forecast.arch import RegionalClimaX


In [3]:


target_features = {'t2m': ['z150', 'z200', 'z250', 'z300', 'z400', 'z500', 't100', 't150', 't250',
                           't300', 't400', 't500', 't600', 't700', 'u50', 'r150', 't2m'],  # >0.9
                   'u10': ['t100', 't150', 't700', 'u50', 'u850', 'u925', 'u1000', 'r50', 'r100', 't2m', 'u10'],
                   # >0.75
                   'v10': ['v1000', 'v10', 'v925', 'u1000', 'u10', 'u925'],  # 均值相关的只有三个，加上了方差相关的
                   'msl': ['z1000', 'u50', 'u100', 'u150', 'u200', 'u250', 'u300', 'u400', 'r50',
                           'r100', 'r150', 'r200', 'msl', 'tp'],  # >0.8
                   'tp': ['t100', 't150', 't250', 't300', 't400', 't500', 'u50', 'r50', 'r100',
                          'r150', 'r200', 'r850', 'r925', 'r1000', 'msl', 'tp']  # >0.8
                   }
# 把字典的所有值合并成一个没有重复元素的集合
all_features = list(set(sum(target_features.values(), [])))
target = ['t2m', 'u10', 'v10', 'msl', 'tp']
for i in target:
    all_features.remove(i)
    all_features.append(i)

data = torch.load('../../data/2011.pt')



class ERA5(Dataset):
    def __init__(self, data: torch.tensor, features_name: dict) -> None:
        self.data = data
        self.features = features_name

    def __len__(self):
        return self.data.shape[0] - 1 - 20 + 1

    def __getitem__(self, idx):
        x = self.data[idx].repeat(20, 1, 1, 1)
        y = self.data[idx+1:idx+21, -5:]
        # x = torch.from_numpy(self.data[idx].values).float().repeat(20, 1, 1, 1)
        # y = torch.from_numpy(self.data[idx+1:idx+21, -5:].values).float()

        lead_time = torch.arange(1, 21).float()

        return x, y, lead_time


data_train, data_val = data[:-200], data[-200:]
print(data_train.shape, data_train.dtype)
print(data_val.shape, data_val.dtype)
data_train, data_val = ERA5(data_train, target_features), ERA5(data_val, target_features)


torch.Size([1260, 39, 161, 161]) torch.float32
torch.Size([200, 39, 161, 161]) torch.float32


In [15]:
import torch
from torch.utils.data import DataLoader
x = torch.arange(20).view(5,2,2)
y = torch.arange(20, 40).view(5,2,2)
z = torch.arange(1,21)
print(z.shape)
print(x)
# 每次取一个batch，batch_size=2
for x_sub, y_sub, z_sub in DataLoader(list(zip(x, y, z)), batch_size=2):
    # print(x_sub)
    print(x_sub.shape)
    print(y_sub.shape)
    print(z_sub.shape)
    # print(x_sub)
    # print(y_sub)
    # break

torch.Size([20])
tensor([[[ 0,  1],
         [ 2,  3]],

        [[ 4,  5],
         [ 6,  7]],

        [[ 8,  9],
         [10, 11]],

        [[12, 13],
         [14, 15]],

        [[16, 17],
         [18, 19]]])
torch.Size([2, 2, 2])
torch.Size([2, 2, 2])
torch.Size([2])
torch.Size([2, 2, 2])
torch.Size([2, 2, 2])
torch.Size([2])
torch.Size([1, 2, 2])
torch.Size([1, 2, 2])
torch.Size([1])


In [4]:

def pretrain_one_epoch(epoch, start_step, model, criterion, data_loader, optimizer, loss_scaler, lr_scheduler, min_loss, all_features):
    loss_val = torch.tensor(0., device="cuda")
    count = torch.tensor(1e-5, device="cuda")

    model.train()

    for step, batch in enumerate(data_loader):
        if step < start_step:
            continue

        x, y, lead_time = [x.cuda() for x in batch]
        x, y = x.view(-1, *x.shape[2:]), y.view(-1, *y.shape[2:])
        lead_time = lead_time.view(-1)


        with torch.cuda.amp.autocast():
            out = model(x, lead_time, all_features)
            loss = criterion(out, y)
            if torch.isnan(loss).int().sum() == 0:
                count += 1
                loss_val += loss

        loss_scaler.scale(loss).backward()
        loss_scaler.step(optimizer)
        loss_scaler.update()
        optimizer.zero_grad()
        if loss.item() < min_loss:
            save_model(model, epoch, step+1, optimizer, lr_scheduler, loss_scaler, min_loss, SAVE_PATH/'backbone_batch_latest.pt')

    return loss_val.item() / count.item()


def train(args, default_vars):
    # model = Model(features_name=target_features).cuda()
    model = RegionalClimaX(default_vars, img_size=[160,160], patch_size=4, embed_dim=1024, depth=8,
                 decoder_depth=2, num_heads=16, mlp_ratio=4, drop_path=0.1, drop_rate=0.1).cuda()
    all_features = default_vars
    param_sum, buffer_sum, all_size = getModelSize(model)
    print(f"Number of Parameters: {param_sum}, Number of Buffers: {buffer_sum}, Size of Model: {all_size:.4f} MB")
    optimizer = torch.optim.AdamW(model.parameters(), args.lr, weight_decay=args.weight_decay, betas=(0.9, 0.95))
    loss_scaler = torch.cuda.amp.GradScaler(enabled=True)
    lr_scheduler, _ = create_scheduler(args, optimizer)
    criterion = torch.nn.MSELoss()

    # load data
    train_dataloader = DataLoader(data_train, args.batch_size, num_workers=0, pin_memory=True, drop_last=False, shuffle=True)  # , num_workers=8, pin_memory=True
    val_dataloader = DataLoader(data_val, args.batch_size, num_workers=0, pin_memory=True, drop_last=False)

    # load
    start_epoch, start_step, min_loss = load_model(model, optimizer, lr_scheduler, loss_scaler, SAVE_PATH / ' model.pt')
    # print(torch.load(SAVE_PATH / 'backbone.pt').items())
    # model.load_state_dict(torch.load(SAVE_PATH / 'backbone.pt')['model'])
    # start_epoch, start_step = 1, 1
    # min_loss = np.inf
    print(f"Start pretrain for {args.pretrain_epochs} epochs")

    for epoch in tqdm(range(start_epoch, args.pretrain_epochs)):
        t0 = time.time()
        train_loss = pretrain_one_epoch(epoch, start_step, model, criterion, train_dataloader, optimizer, loss_scaler,
                                        lr_scheduler, min_loss, all_features)
        t1 = time.time()
        start_step = 0
        lr_scheduler.step(epoch)

        val_loss = climax_pretrain_evaluate(val_dataloader, model, criterion, all_features)

        # if rank == 0 and local_rank == 0:
        print(f"Epoch {epoch} | Train loss: {train_loss:.6f}, Val loss: {val_loss:.6f}, Time: {t1-t0:.2f}s")
        if val_loss < min_loss:
            min_loss = val_loss
            save_model(model, path=SAVE_PATH / 'backbone.pt', only_model=True)
            save_model(model, epoch + 1, 0, optimizer, lr_scheduler, loss_scaler, min_loss, SAVE_PATH / 'backbone_latest.pt')
        save_model(model, epoch + 1, 0, optimizer, lr_scheduler, loss_scaler, min_loss, SAVE_PATH / 'individual_model.pt')



In [5]:

class AttrDict(dict):
    def __getattr__(self, name):
        if name in self:
            return self[name]
        raise AttributeError(f"'AttrDict' object has no attribute '{name}'")

    def __setattr__(self, name, value):
        self[name] = value

def get_args():
    # default_vars, img_size=..., patch_size=2, embed_dim=1024, depth=8, decoder_depth=2, \
    #     num_heads=16, mlp_ratio=4, drop_path=0.1, drop_rate=0.1

    args = dict(batch_size=4, pretrain_epochs=10, fintune_epochs=25, drop=0.0, drop_path=0.1,
                opt='adamw', opt_eps=1e-08, opt_betas=None, clip_grad=1, momentum=0.9, weight_decay=0.05, sched='cosine',
                lr=0.0005, lr_noise=None, lr_noise_pct=0.67, lr_noise_std=1.0, warmup_lr=1e-06, min_lr=1e-05,
                decay_epochs=30, warmup_epochs=5, cooldown_epochs=10, patience_epochs=10, decay_rate=0.1,
                color_jitter=0.4, aa='rand-m9-mstd0.5-inc1', smoothing=0.1, train_interpolation='bicubic',
                repeated_aug=False, reprob=0, remode='pixel', recount=1, resplit=False, fno_bias=False, fno_blocks=4,
                fno_softshrink=0.0, double_skip=False, tensorboard_dir=None, hidden_size=256, num_layers=12,
                checkpoint_activations=False, autoresume=False, num_attention_heads=1, ls_w=4, ls_dp_rank=16)
    args = AttrDict(args)
    return args

args = get_args()
train(args, all_features)

Number of Parameters: 110069360, Number of Buffers: 0, Size of Model: 419.8813 MB
Start pretrain for 10 epochs


  0%|          | 0/10 [02:29<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 19.04 GiB (GPU 0; 8.00 GiB total capacity; 20.24 GiB already allocated; 0 bytes free; 20.26 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [75]:

import os
import torch
import numpy as np
from utils.tools import getModelSize, load_model, save_model

# 把数据一次全部读取，放入dataset和dataloader
data = []
for idx in range(300):
    idx = str(idx).zfill(3)     # 将idx转换成3位数的字符串，在前面补0
    data.append(torch.load(f'../../data/weather_round1_test/input/{idx}.pt', map_location='cpu'))
# data.append(torch.load(f'../../data/weather_round1_test/input/000.pt', map_location='cpu'))

target = ['t2m', 'u10', 'v10', 'msl', 'tp']

model = RegionalClimaX(target, img_size=[160,160], patch_size=4, embed_dim=1024, depth=8,
                 decoder_depth=2, num_heads=16, mlp_ratio=4, drop_path=0.1, drop_rate=0.1)
model.load_state_dict(torch.load('backbone.pt')['model'])
model.cuda()



if not os.path.exists('output'):
    os.mkdir('output')

# inference
model.eval()
for i in range(300):
    # output = []
    idx = str(i).zfill(3)
    data_in = data[i].cuda()
    data_in = data_in[-1:].repeat(20, 1, 1, 1)
    lead_time = torch.arange(1, 21).float().cuda()
    output = model(data_in, lead_time, target).detach().cpu().astype(torch.float16)
    # for j in range(20):
    #     data_out = model(data_in[-1:], torch.tensor([j+1]).float().cuda(), target)
    #     output.append(data_out.cpu().detach().numpy())
    # output = np.concatenate(output, axis=0)
    # output = torch.tensor(output, dtype=torch.float16)
    print(output.shape, output.dtype)
    torch.save(output, f'./output/{idx}.pt')
    print(f'save {idx}.pt ...')

save 000.pt ...
save 001.pt ...
save 002.pt ...
save 003.pt ...
save 004.pt ...
save 005.pt ...
save 006.pt ...
save 007.pt ...
save 008.pt ...
save 009.pt ...
save 010.pt ...
save 011.pt ...
save 012.pt ...
save 013.pt ...
save 014.pt ...
save 015.pt ...
save 016.pt ...
save 017.pt ...
save 018.pt ...
save 019.pt ...
save 020.pt ...
save 021.pt ...
save 022.pt ...
save 023.pt ...
save 024.pt ...
save 025.pt ...
save 026.pt ...
save 027.pt ...
save 028.pt ...
save 029.pt ...
save 030.pt ...
save 031.pt ...
save 032.pt ...
save 033.pt ...
save 034.pt ...
save 035.pt ...
save 036.pt ...
save 037.pt ...
save 038.pt ...
save 039.pt ...
save 040.pt ...
save 041.pt ...
save 042.pt ...
save 043.pt ...
save 044.pt ...
save 045.pt ...
save 046.pt ...
save 047.pt ...
save 048.pt ...
save 049.pt ...
save 050.pt ...
save 051.pt ...
save 052.pt ...
save 053.pt ...
save 054.pt ...
save 055.pt ...
save 056.pt ...
save 057.pt ...
save 058.pt ...
save 059.pt ...
save 060.pt ...
save 061.pt ...
save 062

In [9]:
print(data_in.shape)

torch.Size([1, 2, 70, 161, 161])
